# Simulation Playground

This is an interactive environment to see how the data can be formulted and to test the phenotype simulator.

$$ \textrm{Genetic Risk Score}_{p} = \Sigma^{N}_{i=1} \gamma_{ij}\beta_{i}^{p} +\epsilon_{p} \\
\textrm{p = patient index} \ \  \textrm{N = number of Causal Snps} \ \ \textrm{j = index of interacting snp partner} \\
\gamma=\textrm{Interaction Coefficient} \ \ \beta=\textrm{Effect Size} \ \ \epsilon=\textrm{Optional Patient Specfic Bias}
$$


$$  \textrm{G} = \frac{\textrm{G} - \overline{G}}{\sigma_{G}} $$

$$  \textrm{G'}_{p} = h\textrm{G}_{p} + \sqrt{1-h^{2}}\mathcal{N}(0,1) \\ 
\textrm{h = Heritability Constant} $$

In [1]:
import os, sys
import pandas as pd
import numpy as np

In [2]:
# Define your size parameters
num_snps = 1000
num_genes = 100
people = 100

In [3]:
# Generate the SNPs
feature_ids = np.random.randint(num_genes, size=num_snps)
positions = np.arange(10000,10000 + num_snps)
allele_1 = [np.random.choice(['A', 'C']) for _ in range (num_snps)]
allele_2 = [np.random.choice(['T', 'G']) for _ in range (num_snps)]

snplist = pd.DataFrame(list(zip([0]*len(feature_ids),feature_ids, positions,allele_1,allele_2)), columns = ['Chromosome','Feature ID', 'Position', 'Allele 1', 'Allele 2'])
snplist

,Chromosome,Feature ID,Position,Allele 1,Allele 2
0,0,1,10000,A,T
1,0,33,10001,A,G
2,0,89,10002,C,T
3,0,84,10003,A,T
4,0,44,10004,A,G
...,...,...,...,...,...
995,0,87,10995,A,G
996,0,57,10996,C,T
997,0,84,10997,C,T
998,0,60,10998,A,T


In [4]:
# Generate the genotypes
genotype=[]
for i in range(people):
    person = np.random.binomial(2, 0.3, num_snps)
    genotype.append(person)
df = pd.DataFrame(genotype)
risk_allele = [1 if df.iloc[:,idx].value_counts().to_list()[0] > df.shape[0]//2 else 0 for idx in range(0, df.shape[1])]

In [5]:
snplist['Risk Allele'] = risk_allele
snplist

,Chromosome,Feature ID,Position,Allele 1,Allele 2,Risk Allele
0,0,1,10000,A,T,0
1,0,33,10001,A,G,0
2,0,89,10002,C,T,0
3,0,84,10003,A,T,1
4,0,44,10004,A,G,0
...,...,...,...,...,...,...
995,0,87,10995,A,G,1
996,0,57,10996,C,T,0
997,0,84,10997,C,T,0
998,0,60,10998,A,T,0


In [6]:
# save snplist
snplist.to_csv("sample_data/snplist_{}.csv".format("chr0_test"), sep=" ", index_label=False)

In [7]:
# Format the genotype
matrix = pd.DataFrame(np.array(genotype))
matrix.shape # Person X SNPS
matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,1,0,0,0,1,0,...,0,1,1,0,1,1,1,1,0,0
1,0,1,0,1,0,1,0,0,1,0,...,0,1,0,1,0,0,1,1,0,0
2,0,1,1,1,1,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,1
3,0,1,0,0,0,0,1,0,2,0,...,0,1,0,1,2,1,1,0,0,1
4,1,0,0,0,1,0,0,0,1,0,...,2,0,2,0,1,1,0,0,1,0


In [8]:
# save the genotype
matrix.to_csv("sample_data/genotype_{}.csv".format("chr0_test"), sep=" ", index_label=False) 

In [9]:
import tables
import tqdm
f = tables.open_file("./sample_data/" + "genotype_{}.h5".format("chr0_test"), mode='w')
num_pat = matrix.shape[0]
num_feat = matrix.shape[1]
array_c = f.create_earray(f.root, 'data', tables.IntCol(), (0,num_feat), expectedrows=num_pat,filters=tables.Filters(complib='zlib', complevel=1))
f.close()
f = tables.open_file("./sample_data/" + "genotype_{}.h5".format("chr0_test"),  mode='a')
for pat in tqdm.tqdm(range(num_pat)):
    a = matrix.iloc[pat,:].to_numpy()
    a=np.reshape(a, (1,-1))
    f.root.data.append(a)
f.close()

100%|██████████| 100/100 [00:00<00:00, 2933.24it/s]


In [10]:
# Simulate phenotypes with default parameters
!gepsi phenotype -dp ./sample_data/ --data_identifier chr0_test --phenotype_experiment_name playground_example

Main:  Namespace(case_frac=0.5, causal_gene_cut=0.05, causal_snp_mode='gene', config=None, data_identifier='chr0_test', data_path='./sample_data/', dominance_frac=0.1, heritability=0.8, interactive_cut=0.2, mask_rate=0.1, max_gene_risk=5.0, max_interaction_coeff=2.0, mode='phenotype', n_causal_snps=100, phenotype_experiment_name='playground_example', recessive_frac=0.1, stratify=False)
SNPs: 1000 People: 100
100 Phenotype Scores Closed
Success!


In [11]:
import pickle
causal_gene_name = "causal_genes_chr0_test_playground_example.pkl" 
causal_snp_name = "causal_snp_idx_chr0_test_playground_example.pkl" 
interactive_snp_name = "interactive_snps_chr0_test_playground_example.pkl"
effect_size_name = "effect_size_chr0_test_playground_example.pkl"
phenotype_name = "phenotype_chr0_test_playground_example.pkl"
info_path = "./sample_data/"
with open((info_path + causal_snp_name), 'rb') as f:
            causal_snps = pickle.load(f)
with open((info_path + interactive_snp_name), 'rb') as f:
            interactive_snps = pickle.load(f)
with open((info_path + causal_gene_name), 'rb') as f:
            causal_genes = pickle.load(f)
with open((info_path + effect_size_name), 'rb') as f:
            effect_sizes = pickle.load(f)
with open((info_path + phenotype_name), 'rb') as f:
            phenotype = pickle.load(f)

In [12]:
phenotype

[0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1]